In [1]:
import os
import pandas as pd

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from statistics import mean, median

%load_ext jupyter_black

## Functions ⨏

### 🧩  Functions: general & data loading

#### Function to get pathnames ⨏🏔

In [2]:
# function to get the names of file paths
def get_file_paths(root_folder, filename):
    file_paths = []
    for folder_name, subfolders, filenames in os.walk(root_folder):
        for fname in filenames:
            if fname == filename:
                file_paths.append(os.path.join(folder_name, fname))

    return file_paths

#### Function to extract only desired data ⨏🛢 NB THIS VERSION INCLUDES UP TO 5 WORMS

In [3]:
def import_extract(filepath_metadata_featuresN):
    # print("filepath_metadata_featuresN: ", filepath_metadata_featuresN)
    # import data

    timedata = pd.read_hdf(filepath_metadata_featuresN, "/timeseries_data")
    trajdata = pd.read_hdf(filepath_metadata_featuresN, "/trajectories_data")

    # if there is no manual wormindex then keep wormindex from timeseries,
    # and use was_skeletonized to put into usual "has skeleton" column
    if "worm_index_manual" not in trajdata.columns:
        trajectories = pd.DataFrame(
            {
                "worm_id": timedata["worm_index"],
                "has_skeleton": trajdata["was_skeletonized"],
                "frame_number": trajdata["frame_number"],
                "x": trajdata["coord_x"],
                "y": trajdata["coord_y"],
            }
        )

    # if there is a manual worm index in traj file then keep that instead of wormindex
    else:
        trajectories = pd.DataFrame(
            {
                "worm_id": trajdata["worm_index_manual"],
                "has_skeleton": trajdata["has_skeleton"],
                "frame_number": trajdata["frame_number"],
                "x": trajdata["coord_x"],
                "y": trajdata["coord_y"],
            }
        )
    trajectories = pd.concat(
        [
            trajectories,
            timedata[timedata.iloc[:, 3:].dropna(axis="columns", how="all").columns],
        ],
        axis=1,
    )

    # keep only the data that has good skeletons
    trajectories = trajectories[trajectories["has_skeleton"] == 1]

    # from that, keep only the data corresponding to the first two worm ids, which are the 2 good ones
    trajectories = trajectories[
        trajectories["worm_id"].isin(trajectories["worm_id"].unique()[0:3])
    ]

    # from that, keep only data of those ids if they have at least 100 frames
    frame_cutoff = 100
    longtrajectories = (
        pd.DataFrame()
    )  # this will contain only the trajectories if they pass the longer than framecutoff rule
    for i in range(
        len((trajectories["worm_id"].unique()[0:2]))
    ):  # cycle through 0 to 1 or just 0 if only one index
        a = trajectories["worm_id"].unique()[
            i
        ]  # get object that is of the wormid column only those = first index
        length_a = (trajectories["worm_id"] == a).sum()  # check how long it is
        if length_a > frame_cutoff:  # if a is greater than framecutoff then keep it
            longtrajectories = pd.concat(
                [
                    longtrajectories,
                    trajectories[
                        trajectories["worm_id"] == trajectories["worm_id"].unique()[i]
                    ],
                ]
            )

    return longtrajectories  # which is actutruey mostly timeseries df but have inserted some things from trajectories df

#### Function to organise loading true data ⨏🗄

In [4]:
# input args: df with pathnames
# output is
# bigdict = {"fpath": {"cond": "sexc", "data": pd.DataFrame()}}
# bigdf = one big data frame will true worms true conditions.
# has added column "cond" (mock/avsv/sexc) and "filename" (corresponds to videoname)


def organise_data(pathname_dict):
    bigdf = pd.DataFrame()
    bigdict = dict()
    for key, val in pathname_dict.items():
        for i in range(len(pathname_dict[key])):
            # assign pathname to key in new bigdict
            bigdict[pathname_dict[key][i]] = dict()
            # save cond within bigdict type cond
            bigdict[pathname_dict[key][i]]["cond"] = key
            # load and save data within bigdict type data
            bigdict[pathname_dict[key][i]]["data"] = import_extract(
                pathname_dict[key][i],
            )
            # access dataframes within data type and create new columns for name and for cond
            bigdict[pathname_dict[key][i]]["data"]["cond"] = key
            bigdict[pathname_dict[key][i]]["data"]["filename"] = pathname_dict[key][i]

            # now append this new dataframe into dataframe with true data
            bigdf = pd.concat([bigdf, bigdict[pathname_dict[key][i]]["data"]])

    return bigdf, bigdict

#### Function to check consecutive numbers ⨏ 1 2 3#

In [5]:
def consecutive(data, stepsize=1):
    return np.split(data, np.where(np.diff(data) != stepsize)[0] + 1)

### 🔍 Functions: find & clean reversals, forward runs, and omegas

#### Function to find reversals ⨏🔎↩️

In [6]:
def findreversals(worm_frame_speed):
    revStartInd = []  # empty lists
    revEndInd = []

    reversals = worm_frame_speed.index[worm_frame_speed["speed"] < 0]
    # print("reversals: ", reversals)
    # reversal is list of indexs of all frames with neg speed

    # first get two lists with start and end idexes of reversals
    # then will check within reversal, if there is jump in frame split in two reversals,
    # if there are less than 15 nans and neg then count as one

    # go through each element of reversals ie all the indexes with neg speed
    # and pull out start and end indices
    for r in reversals:
        # print("r is", r)
        #### now start going through and checking
        # count first revindex as start of reversal,
        # if next is pos/nan also count as end of reversal
        if r == np.min(reversals):
            revStartInd.append(r)
            if worm_frame_speed.loc[r + 1]["speed"] >= 0 or pd.isna(
                worm_frame_speed.loc[r + 1]["speed"]
            ):
                revEndInd.append(r)
                # print("appended", r, "to revendind")

        # if the index is not 0 and is not last check whether previous&last indexs have positive speed
        elif (r > np.min(reversals)) & (r < np.max(reversals)):
            #####CHECK PREV ROW#####
            # if speed in prev row is greater than 0
            if worm_frame_speed.loc[r - 1]["speed"] >= 0 or pd.isna(
                worm_frame_speed.loc[r - 1]["speed"]
            ):
                revStartInd.append(r)
                # print("appended", r, "to revstartind")

            #####CHECK NEXT ROW#####
            # if speed in next row is greater than 0 or nan
            if worm_frame_speed.loc[r + 1]["speed"] >= 0 or pd.isna(
                worm_frame_speed.loc[r + 1]["speed"]
            ):
                revEndInd.append(r)
                # print("appended", r, "to revendind")

        # if neg speed at last negative speed index count it as end of last reversal
        # if prev was positive/nan count as start of reversal
        elif r == np.max(reversals):
            revEndInd.append(r)

            if worm_frame_speed.loc[r - 1]["speed"] >= 0 or pd.isna(
                worm_frame_speed.loc[r - 1]["speed"]
            ):
                revStartInd.append(r)
                # print("appended", r, "to revstartind")

    # check same number of rev start and end
    if len(revStartInd) != len(revEndInd):
        print("PROBLEM: revstart and revend indices don't match up :O find rev")

    return revStartInd, revEndInd

#### Function to clean reversals + calculate length ⨏🛁↩️ + 📏↩️

In [7]:
# now clean reversal start/end indices and calculate reversal lengths
def cleanreversals(worm_frame_speed, revStartInd, revEndInd):
    # create empty lists to hold output
    clean_revStartInd = []
    clean_revEndInd = []

    # first check frames within this reversal
    for l in range(len(revStartInd)):
        ##first assess whether there are any frames missing in this reversal
        # print("l", l)
        # print("len revEndInd", len(revEndInd))
        # print("len revStartInd", len(revStartInd))
        # length in index counts of this reversal
        this_revlenI = revEndInd[l] - revStartInd[l]

        # print("revEndInd[l]", revEndInd[l])
        # print("revStartInd[l]", revStartInd[l])

        # length in frames of this reversal
        this_revlenF = (
            worm_frame_speed.loc[revEndInd[l], "frame_number"]
            - worm_frame_speed.loc[revStartInd[l], "frame_number"]
        )

        # append start and end indices of reversal to clean dfs, then see if need to add any
        clean_revStartInd.append(revStartInd[l])
        clean_revEndInd.append(revEndInd[l])

        # if frame and index count dont match then check how many frames missing and if consecutive
        if this_revlenI != this_revlenF:
            # do consec analysis
            # get list of frame numbers between those two indexes
            this_rev_frames = worm_frame_speed.loc[
                revStartInd[l] : revEndInd[l], "frame_number"
            ].tolist()
            rev_cc_array = consecutive(this_rev_frames)

            for i in range(len(rev_cc_array)):
                # print("i is cycle inside len of cc array", i)
                # print("rev_cc_array[i]", rev_cc_array[i])
                # start at i=1 because first reversal break happens between 0 and 1
                if i > 0:
                    revbreakend = rev_cc_array[i][0]
                    revbreakstart = rev_cc_array[i - 1][-1]
                    revbreaklen = revbreakend - revbreakstart
                    # if reversal breakend is greater than max_frames_missing, append the frame of where rev break starts to be new revend ind
                    # and frame of where rev break ends to be new revstart ind
                    if revbreaklen > max_frames_missing + 1:
                        clean_revEndInd.append(
                            worm_frame_speed[
                                worm_frame_speed["frame_number"] == revbreakstart
                            ].index[0]
                        )
                        clean_revStartInd.append(
                            worm_frame_speed[
                                worm_frame_speed["frame_number"] == revbreakend
                            ].index[0]
                        )

    # now check frames between reversals
    for l in range(len(revStartInd)):
        # dont need to check what came before first reversal, is definitely rev start
        # if is 2nd revstart or later, check interval between previous rev end and this
        # want only frames that are in interval hence +1 and -1 after the revendind and revstartind
        if l > 0:
            # pull out interval of reversal (includes revstart and revend)
            this_interval = worm_frame_speed.loc[
                (revEndInd[l - 1]) : (revStartInd[l]), ["frame_number", "speed"]
            ]
            this_intervalS = this_interval["speed"]
            # get the the number of frames between last revend and this revstart
            this_intervalF = this_interval["frame_number"]
            this_intervalF = this_intervalF.iloc[-1] - this_intervalF.iloc[0]
            # if interval between reversals is all nans AND 4 frames or less
            if all(this_intervalS.iloc[1:-1].isna()) and this_intervalF <= (
                max_frames_missing + 1
            ):
                # then get rid of revEndInd[l - 1] and revStartInd[l]
                clean_revStartInd.remove(revStartInd[l])
                clean_revEndInd.remove(revEndInd[l - 1])

    # sort lists to make sure added frames are true in order
    clean_revStartInd.sort()
    clean_revEndInd.sort()

    # check same number of rev start and end
    if len(clean_revStartInd) != len(clean_revEndInd):
        print("PROBLEM: revstart and revend indices don't match up :O")

    # calculate reversal length
    revstart_frames = worm_frame_speed.loc[clean_revStartInd, "frame_number"].tolist()
    revend_frames = worm_frame_speed.loc[clean_revEndInd, "frame_number"].tolist()
    rev_lengths = np.array(revend_frames) - np.array(revstart_frames)
    rev_lengths += 1
    rev_lengths = rev_lengths.tolist()

    # print("revStartInd", revStartInd)
    # print("revEndInd", revEndInd)
    # print("clean_revStartInd", clean_revStartInd)
    # print("clean_revEndInd", clean_revEndInd)
    # print("rev_lengths", rev_lengths)

    return clean_revStartInd, clean_revEndInd, rev_lengths

#### Function to find fwd runs ⨏🔎⬆️

In [8]:
def findfwdruns(worm_frame_speed):
    # empty lists
    fwdStartInd = []
    fwdEndInd = []

    # NB forwards contains all indexes with positive or 0 speed, whereas reversals contains only indexes with neg speed
    forwards = worm_frame_speed.index[worm_frame_speed["speed"] >= 0]

    # print("forwards: ", forwards)
    # forwards is list of indexes of all frames with neg speed

    # first get two lists with start and end idexes of forward runs
    # then will check within forward run, if there is jump in frame split in two forward runs,
    # if there are less than 15 nans and pos speed again then count as one

    # go through each element of forwards ie all the indexes with pos or 0 speed
    # and pull out start and end indices

    for r in forwards:
        # print("r is", r)
        #### now start going through and checking
        # count first fwdindex as start of fwdrun,
        # then check if there is a next - could be first and last, in which case append also to endind
        # if there is next and is neg/nan also count as end of fwdrun (dont count as end of fwd run if its 0)
        if r == np.min(forwards):
            fwdStartInd.append(r)
            # print("appended", r, "to fwdstartind")

            if r == np.max(forwards):
                fwdEndInd.append(r)
            elif r < np.max(forwards):
                if worm_frame_speed.loc[r + 1]["speed"] < 0 or pd.isna(
                    worm_frame_speed.loc[r + 1]["speed"]
                ):
                    fwdEndInd.append(r)
                    # print("appended", r, "to fwdendind")

        # if the index is not 0 and is not last check whether previous&last indexs have negative speed
        elif (r > np.min(forwards)) & (r < np.max(forwards)):
            #####CHECK PREV ROW#####
            # if speed in prev row is smaller than 0 or nan
            if worm_frame_speed.loc[r - 1]["speed"] < 0 or pd.isna(
                worm_frame_speed.loc[r - 1]["speed"]
            ):
                fwdStartInd.append(r)
                # print("appended", r, "to fwdstartind")

            #####CHECK NEXT ROW#####
            # if speed in next row is smaller than 0 or nan
            if worm_frame_speed.loc[r + 1]["speed"] < 0 or pd.isna(
                worm_frame_speed.loc[r + 1]["speed"]
            ):
                fwdEndInd.append(r)
                # print("appended", r, "to fwdendind")

        # if pos speed at last forward speed index, count it as end of last fwd run
        # if prev was negative/nan count also as start of fwd run
        elif r == np.max(forwards):
            fwdEndInd.append(r)
            # print("appended", r, "to fwdendind")

            if worm_frame_speed.loc[r - 1]["speed"] < 0 or pd.isna(
                worm_frame_speed.loc[r - 1]["speed"]
            ):
                fwdStartInd.append(r)
                # print("appended", r, "to fwdstartind")

    # check same number of rev start and end
    if len(fwdStartInd) != len(fwdEndInd):
        print(
            "PROBLEM: fwdstart and fwdend indices don't match up :O (find fwdruns function)"
        )

    # print("fwdStartInd", fwdStartInd)
    # print("fwdEndInd", fwdEndInd)

    return fwdStartInd, fwdEndInd

#### Function to clean fwd runs + calculate length ⨏🛁⬆️ + 📏⬆

In [9]:
# clean forward run start/end indices and calculate fwdrun lengths
def cleanfwdruns(worm_frame_speed, fwdStartInd, fwdEndInd):
    # create empty lists to hold output
    clean_fwdStartInd = []
    clean_fwdEndInd = []

    # first check frames within this fwd run
    for l in range(len(fwdStartInd)):
        ##first assess whether there are any frames missing in this fwdrun
        # print("l", l)
        # print("len fwdEndInd", len(fwdEndInd))
        # print("len fwdStartInd", len(fwdStartInd))
        # length in index counts of this fwdrun
        this_fwdlenI = fwdEndInd[l] - fwdStartInd[l]

        # print("fwdEndInd[l]", fwdEndInd[l])
        # print("fwdStartInd[l]", fwdStartInd[l])

        # length in frames of this fwd run
        this_fwdlenF = (
            worm_frame_speed.loc[fwdEndInd[l], "frame_number"]
            - worm_frame_speed.loc[fwdStartInd[l], "frame_number"]
        )

        # append start and end indices of fwd run to clean dfs, then see if need to add any
        clean_fwdStartInd.append(fwdStartInd[l])
        clean_fwdEndInd.append(fwdEndInd[l])

        # if frame and index count dont match then check how many frames missing and if consecutive
        if this_fwdlenI != this_fwdlenF:
            # do consecutive analysis
            # get list of frame numbers between those two indexes
            this_fwd_frames = worm_frame_speed.loc[
                fwdStartInd[l] : fwdEndInd[l], "frame_number"
            ].tolist()
            fwd_cc_array = consecutive(this_fwd_frames)

            for i in range(len(fwd_cc_array)):
                # print("i is cycle inside len of cc array", i)
                # print("fwd_cc_array[i]", fwd_cc_array[i])
                # start at i=1 because first fwdrun break happens between 0 and 1
                if i > 0:
                    fwdbreakend = fwd_cc_array[i][0]
                    fwdbreakstart = fwd_cc_array[i - 1][-1]
                    fwdbreaklen = fwdbreakend - fwdbreakstart
                    # if fwd run breakend is greater than max_frames_missing, append the frame of where fwd break starts to be new fwdend ind
                    # and frame of where fwd break ends to be new fwdstart ind
                    if fwdbreaklen > max_frames_missing + 1:
                        clean_fwdEndInd.append(
                            worm_frame_speed[
                                worm_frame_speed["frame_number"] == fwdbreakstart
                            ].index[0]
                        )
                        clean_fwdStartInd.append(
                            worm_frame_speed[
                                worm_frame_speed["frame_number"] == fwdbreakend
                            ].index[0]
                        )

    # now check frames between fwd runs
    for l in range(len(fwdStartInd)):
        # dont need to check what came before first fwd run, is definitely fwd start
        # if is 2nd fwd run or later, check interval between previous fwd end and this fwd start
        # want only frames that are in interval hence -1 after the fwdendind
        if l > 0:
            # pull out interval of fwd run (includes fwdstart and fwdend)
            this_interval = worm_frame_speed.loc[
                (fwdEndInd[l - 1]) : (fwdStartInd[l]), ["frame_number", "speed"]
            ]
            this_intervalS = this_interval["speed"]
            # get the the number of frames between last fwdend and this fwdstart
            this_intervalF = this_interval["frame_number"]
            this_intervalF = this_intervalF.iloc[-1] - this_intervalF.iloc[0]
            # if interval between fwd runs is all nans AND 4 frames or less
            if all(this_intervalS.iloc[1:-1].isna()) and this_intervalF <= (
                max_frames_missing + 1
            ):
                # then get rid of fwdEndInd[l - 1] and fwdStartInd[l]
                clean_fwdStartInd.remove(fwdStartInd[l])
                clean_fwdEndInd.remove(fwdEndInd[l - 1])

    # sort lists to make sure added frames are all in order
    clean_fwdStartInd.sort()
    clean_fwdEndInd.sort()

    # check same number of fwd start and end
    if len(clean_fwdStartInd) != len(clean_fwdEndInd):
        print("PROBLEM: fwdstart and fwdend indices don't match up :O")

    # calculate fwd run length
    fwdstart_frames = worm_frame_speed.loc[clean_fwdStartInd, "frame_number"].tolist()
    fwdend_frames = worm_frame_speed.loc[clean_fwdEndInd, "frame_number"].tolist()
    fwd_lengths = np.array(fwdend_frames) - np.array(fwdstart_frames)
    fwd_lengths += 1
    fwd_lengths = fwd_lengths.tolist()

    # print("fwdStartInd", fwdStartInd)
    # print("fwdEndInd", fwdEndInd)
    # print("clean_fwdStartInd", clean_fwdStartInd)
    # print("clean_fwdEndInd", clean_fwdEndInd)
    # print("fwd_lengths", fwd_lengths)

    return clean_fwdStartInd, clean_fwdEndInd, fwd_lengths

#### function to remove reversals too short or too long 🚫↩️👶🏻👨🏼‍🦳

In [10]:
# min and max length for considering as reversals
# minrevlen = 15
# maxrevlen = 80


def exclude_short_long_revs(data_df):
    # make copy of data_df to work on within function
    infunct_data_df = data_df.copy()

    # add allrevstart and allrevend columns to infunct_data_df
    infunct_data_df["TrueRevStart"] = False
    infunct_data_df["TrueRevEnd"] = False

    # will contain rev len lists and unique id list matching length to create new true_rev_lengths
    revlengs_ids = []
    revlengs = []
    true_rev_lengths = []

    wormlist = infunct_data_df["unique_id"].unique()

    # cycle within each worm
    for worm in wormlist:
        # list to hold lists of true reversal lengths
        this_worm_tRLg = []

        this_worm = infunct_data_df[infunct_data_df["unique_id"] == worm].copy()

        # get list with revstart and revend indices = true
        revstarts = this_worm[this_worm["RevStart"] == True].index.tolist()
        revends = this_worm[this_worm["RevEnd"] == True].index.tolist()

        # cycle within the list of revstart = true indices
        for R in range(len(revstarts)):
            # for each revstart, check how many frames between revstart and next revend
            # R is position in list of revstarts
            # fstart is start and end frames of
            fstart = this_worm.loc[revstarts[R], "frame_number"]
            fend = this_worm.loc[revends[R], "frame_number"]
            this_rev_len = fend - fstart + 1

            # if 15<this_rev_len<80
            # add revstart and revend to truerevstart and truerevend columns
            # add revlen to new_rev_lengths
            if this_rev_len > minrevlen and this_rev_len < maxrevlen:
                infunct_data_df.loc[revstarts[R], "TrueRevStart"] = True
                infunct_data_df.loc[revends[R], "TrueRevEnd"] = True

                this_worm_tRLg.append(this_rev_len)

        # to store reversal lengths
        # cycle thrugh list of reversal lengths and append worm id to worm id list and rev leng to rev list
        # then will join both these lists into df
        for k in this_worm_tRLg:
            revlengs_ids.append(worm)
            revlengs.append(k)

    infunct_data_df.rename(
        columns={
            "RevStart": "ogRevStart",
            "RevEnd": "ogRevEnd",
            "TrueRevStart": "RevStart",
            "TrueRevEnd": "RevEnd",
        },
        inplace=True,
    )

    # now create true_rev_lengths
    # thisdf will contain unique worm id (short) as 1st column and the length of each reversal in 2nd column
    true_rev_lengths = pd.DataFrame()
    true_rev_lengths = true_rev_lengths.assign(unique_id=revlengs_ids, revlen=revlengs)

    return infunct_data_df, true_rev_lengths

#### function to correct misidentified heads in reversals ⨏🔄👶🏻🐛

In [11]:
def correct_misheads(corrected_data_df, head_errors_list):
    # add this column which will show whether worm has had head corrected or not
    corrected_data_df["mishead"] = False

    for index, row in head_errors_list.iterrows():
        # get some values specific to this worm
        this_worm = head_errors_list.loc[index, "unique_id_long"]
        print("this_worm ie long", this_worm)
        print("unique id short", head_errors_list.loc[index, "unique_id"])
        this_SF = head_errors_list.loc[index, "StartFrame"]
        this_EF = head_errors_list.loc[index, "EndFrame"]
        # find start and end index of the stretch to be corrected
        startindex = corrected_data_df[
            (corrected_data_df["unique_id_long"] == this_worm)
            & (corrected_data_df["frame_number"] == this_SF)
        ].index[0]
        endindex = corrected_data_df[
            (corrected_data_df["unique_id_long"] == this_worm)
            & (corrected_data_df["frame_number"] == this_EF)
        ].index[0]

        # select this stretch in dataframe and replace with inverse values
        corrected_data_df.loc[startindex:endindex, "speed"] = -corrected_data_df.loc[
            startindex:endindex, "speed"
        ]

        # change value of "mishead" column to true to indicate this worm has had its head misidentified and thus speed corrected
        corrected_data_df.loc[
            corrected_data_df["unique_id"] == this_worm, "mishead"
        ] = True

        print(
            "unique id short from new dataset",
            corrected_data_df.loc[
                (corrected_data_df["unique_id_long"] == this_worm)
                & (corrected_data_df["frame_number"] == this_SF),
                "unique_id",
            ],
        )

    return corrected_data_df

#### function to find (inferred) omega turns ⨏Ω↺

In [12]:
minomegalength = 20
maxomegalength = 400

# draft find omega turns function


def findomegas(data_df):
    # NB omega start frame is last frame of rev, omega end frame is next frame in worm (cant label missing frame!)

    # create 2 new columns in datadf: omegaS, omegaE (all false)
    data_df["OmegaS"] = False
    data_df["OmegaE"] = False

    # create 4 lists: omega_startindex, omegalengths, revlengths (nb revlenghts contains only the lenght of reversals followed by omega), unique_id
    omega_startindex = []
    omega_startframes = []
    omega_lengths = []
    omega_revlengths = []
    omega_ids = []

    # cycle through each worm
    worms = data_df["unique_id"].unique().tolist()
    for worm in worms:
        # print("worm", worm)
        this_worm = data_df[data_df["unique_id"] == worm]

        # within worm, pull out two lists of framenumber at which ogrevstart = true, and ogrevend = true
        thiswormRS = this_worm.index[this_worm["ogRevStart"] == True].tolist()
        thiswormRE = this_worm.index[this_worm["ogRevEnd"] == True].tolist()

        # print("thiswormRS", thiswormRS)
        # print("thiswormRE", thiswormRE)

        # save also the last index corresponding to this worm
        thisworm_maxindex = this_worm.index.max()

        # then cycle through list of OGREVEND frames
        for RE in thiswormRE:
            # print("RE", RE)
            # save frame number
            REframenumber = this_worm.loc[RE, "frame_number"]

            # save remainder of worm speed as smaller df
            speedseries = this_worm.loc[(RE + 1) : thisworm_maxindex, ["speed"]]
            # print("speedseries", speedseries)

            # check if speedseries is empty (this happens if reversal ends on last frame of that worm). only do the rest if is not empty (IE False)
            if speedseries.empty == False:
                # print("speedseries is not empty")
                # check if next first valid index within speedseries (ie next speed that is not nan or missing) is nan
                # if first valid index isnan is False, that means that there is a next non-nan value before end of worm
                if pd.isna(speedseries.first_valid_index()) == False:
                    # get (frame number of that index) - 1 (we want frame of last nan / missing index )
                    nextvalidindex = speedseries.first_valid_index()
                    # print("next valid index exists")

                # if first valid index isnan is True, that means that there is not a non-nan value before end of worm
                # means that na stretch runs up until the end - take last frame of series as the last frame of omega turn
                elif pd.isna(speedseries.first_valid_index()) == True:
                    nextvalidindex = speedseries.index.max()
                    # print("there is no next valid index before worm end")

                # print("nextvalidindex", nextvalidindex)

                # get omega start and end framenumber & index
                # (omega starts at last frame of  rev )
                omegastartframe = this_worm.loc[(RE), "frame_number"]
                omegastartindex = RE

                omegaendframe = this_worm.loc[(nextvalidindex), "frame_number"]
                omegaendindex = nextvalidindex

                # print("omegastartframe", omegastartframe)
                # print("omegaendframe", omegaendframe)

                # omegalength is not plus one bc we have taken start frame to be last frame of rev
                omegalength = omegaendframe - omegastartframe
                # print("omegalength", omegalength)

                # if omegalength is between 20-400 we consider an omega turn (try then with 80)
                if omegalength >= minomegalength and omegalength <= maxomegalength:
                    # add start and end trues to omegaS and omegaE
                    data_df.loc[omegastartindex, "OmegaS"] = True
                    data_df.loc[omegaendindex, "OmegaE"] = True

                    # calculate length of preceeding reversal, from ogrevstart & ogrevend frame got at beggining
                    thisrevnumber = thiswormRE.index(RE)
                    thisrevstartI = thiswormRS[thisrevnumber]
                    thisrevendI = thiswormRE[thisrevnumber]
                    thisrevstartF = this_worm.loc[thisrevstartI, "frame_number"]
                    thisrevsendF = this_worm.loc[thisrevendI, "frame_number"]

                    thisrevlen = thisrevsendF - thisrevstartF + 1

                    # add omega starti, omega length,  revlength and wormid of this omega turn to corresponding 4 lists
                    omega_startindex.append(omegastartindex)
                    omega_startframes.append(omegastartframe)
                    omega_lengths.append(omegalength)
                    omega_revlengths.append(thisrevlen)
                    omega_ids.append(worm)

            # else:
            # print("speedseries is empty")

    omegas = pd.DataFrame(
        list(
            zip(
                omega_ids,
                omega_lengths,
                omega_revlengths,
                omega_startindex,
                omega_startframes,
            )
        ),
        columns=[
            "unique_id",
            "omega_length",
            "omega_revlength",
            "omega_startindex",
            "omega_startframe",
        ],
    )
    # print("omega_lengths", omega_lengths)
    # print("omega_revlengths", omega_revlengths)
    # print("omega_ids", omega_ids)

    return omegas

### 🔪Functions: data cleaning (general)

#### function to remove worms with scant frames ⨏🚫💩🐛

### 🔪Functions: data cleaning (general)

#### function to remove worms with scant frames ⨏🚫💩🐛

In [13]:
def remove_shitty_worms(data_df):
    wormlist = data_df["unique_id"].unique()
    max_frames = data_df["frame_number"].max()
    good_worms = []
    for worm in wormlist:
        this_worm_speed = data_df["speed"][data_df["unique_id"] == worm]
        this_worm_gf = this_worm_speed.count()
        this_worm_gf_percent = (this_worm_gf / max_frames) * 100
        if this_worm_gf_percent > min_good_frames_percent:
            good_worms.append(worm)

    updated_data_df = data_df[data_df["unique_id"].isin(good_worms)].copy()

    return updated_data_df

### 🤓Functions: extract data parameters

#### function to count good frames in each worm ⨏✅🎞

In [14]:
# function to check missing data
# input is data_df
# output is df with uniqueid, total frames that are not nan, percentage, and color code


def count_good_frames(data_df):
    gf_per_worm = pd.DataFrame(pd.Series(wormlist, name="unique_id"))
    gf_per_worm["good_frames"] = pd.NA

    # add column with color corresponding to condition (for plotting)
    # create lists with unique_ids of each worm condition
    mock_worms = data_df["unique_id"][data_df["cond"] == "mock"].unique().tolist()
    avsv_worms = data_df["unique_id"][data_df["cond"] == "avsv"].unique().tolist()

    for worm in wormlist:
        this_worm_speed = data_df["speed"][data_df["unique_id"] == worm]
        this_worm_gf = this_worm_speed.count()
        gf_per_worm.loc[gf_per_worm["unique_id"] == worm, "good_frames"] = this_worm_gf

        if worm in mock_worms:
            gf_per_worm.loc[gf_per_worm["unique_id"] == worm, "color_cond"] = "#a8a9a8"
            gf_per_worm.loc[gf_per_worm["unique_id"] == worm, "cond"] = "mock"

        elif worm in avsv_worms:
            gf_per_worm.loc[gf_per_worm["unique_id"] == worm, "color_cond"] = "#87da4b"
            gf_per_worm.loc[gf_per_worm["unique_id"] == worm, "cond"] = "avsv"

    # add column with percentage of good frames per worm
    max_frames = data_df["frame_number"].max()
    gf_per_worm["percent_good"] = (gf_per_worm["good_frames"] / max_frames) * 100
    gf_per_worm = gf_per_worm.reindex(
        columns=["color_cond", "cond", "unique_id", "good_frames", "percent_good"]
    )

    return gf_per_worm
    # gf_per_worm is dataframe with 4 cols:
    # unique_id,
    # number of good frames that worm has,
    # percentage of good frames (frames / max frames across true conds),
    # color code corresponding to condition

#### function to calculate feats. percentage per worm: FWD and BCK movement ⨏↩️🐛

In [15]:
def calculate_fbpercent(gf_per_worm, data_df):
    fwdback_perworm = gf_per_worm.copy()
    fwdback_perworm["fwdframes"] = 0
    fwdback_perworm["bckframes"] = 0
    fwdback_perworm["totalframes"] = 0

    for worm in wormlist:
        this_worm = data_df[data_df["unique_id"] == worm]
        fwdback_perworm.loc[worm, "fwdframes"] = len(this_worm[this_worm["fw"] == True])
        fwdback_perworm.loc[worm, "bckframes"] = len(this_worm[this_worm["bk"] == True])
        fwdback_perworm.loc[worm, "totalframes"] = (
            fwdback_perworm.loc[worm, "fwdframes"]
            + fwdback_perworm.loc[worm, "bckframes"]
        )

    fwdback_perworm["percent_fw"] = (
        fwdback_perworm["fwdframes"] / fwdback_perworm["totalframes"]
    ) * 100
    fwdback_perworm["percent_bk"] = (
        fwdback_perworm["bckframes"] / fwdback_perworm["totalframes"]
    ) * 100

    return fwdback_perworm

### 🎨 Functions: plotting

#### function to organise traj data to call plotter function ⨏🗺📍


In [16]:
def organise_indv_traj_plot(all_data, outputdir):
    # set some constants
    min_x = 0
    max_x = 46000
    min_y = 0
    max_y = 46000
    fig, ax = plt.subplots()
    # cycle through all worms
    for worm in wormlist:
        this_worm = all_data[all_data["unique_id"] == worm]
        cond = all_data["cond"][all_data["unique_id"] == worm].unique().tolist()[0]
        wormstr = np.array2string(worm)
        thisx = this_worm["coord_x_body"]
        thisy = this_worm["coord_y_body"]
        thisframe = this_worm["frame_number"]
        this_filename = outputdir + cond + wormstr

        this_cmap = "viridis"
        # now plot trajectory of this worm using above variables

        plot_traj(
            thisx,
            thisy,
            thisframe,
            this_cmap,
            min_x,
            max_x,
            min_y,
            max_y,
            this_filename,
        )
    return

#### function to make pretty trajectory plots ⨏🎨📍

In [17]:
def plot_traj(xvals, yvals, frames, this_cmap, minx, maxx, miny, maxy, filename):
    fig, ax = plt.subplots()
    ax.scatter(xvals, yvals, s=0.1, c=frames, cmap=this_cmap, marker=".")
    ax.set_xlim(left=minx, right=maxx)
    ax.set_ylim(bottom=miny, top=maxy)

    plt.savefig(
        filename,
        transparent=True,
    )
    plt.show()

    return

In [18]:
def organise_indv_traj_plot(true_data, outputdir):
    # set some constants
    min_x = 0
    max_x = 46000
    min_y = 0
    max_y = 46000
    fig, ax = plt.subplots()
    # cycle through true worms
    for worm in wormlist:
        this_worm = true_data[true_data["unique_id"] == worm]
        cond = true_data["cond"][true_data["unique_id"] == worm].unique().tolist()[0]
        wormstr = np.array2string(worm)
        thisx = this_worm["coord_x_body"]
        thisy = this_worm["coord_y_body"]
        thisframe = this_worm["frame_number"]
        this_filename = outputdir + cond + wormstr

        this_cmap = "viridis"
        # now plot trajectory of this worm using above variables

        plot_traj(
            thisx,
            thisy,
            thisframe,
            this_cmap,
            min_x,
            max_x,
            min_y,
            max_y,
            this_filename,
        )
    return

## Inputs 🥚🧈🫐



In [19]:
## only inputs needed for the whole thingymawhatsits to run

#####################################################################


# 1. set output directory (need to make it match name of legit existing folder)
outputdirhim5 = "/home/Documents/neuroUCL/phd/current/project/tierpsy stuff for tracking and reversals/duringconditioning/him-5/feb2/"
outputdirBAR302 = "/home/Documents/neuroUCL/phd/current/project/tierpsy stuff for tracking and reversals/BAR302/11jan/"

# 2. set code to choose true other parameters and suffix (HIM-5 / BAR302)
code = "HIM-5"
# code = "BAR302"


####################################################################


if code == "HIM-5":
    outputdir = outputdirhim5

    folders = [
        "/home/Documents/neuroUCL/phd/current/project/tierpsy stuff for tracking and reversals/data_duringcond/Results/mock",
        "/home/Documents/neuroUCL/phd/current/project/tierpsy stuff for tracking and reversals/data_duringcond/Results/avsv",
    ]

elif code == "BAR302":
    outputdir = outputdirBAR302

    # folders = [
    #     "/home/Documents/neuroUCL/phd/current/project/tierpsy stuff for tracking and reversals/data_uptoDEC/Results/BAR302_aftercond/mock",
    #     "/home/Documents/neuroUCL/phd/current/project/tierpsy stuff for tracking and reversals/data_uptoDEC/Results/BAR302_aftercond/avsv",
    #     "/home/Documents/neuroUCL/phd/current/project/tierpsy stuff for tracking and reversals/data_uptoDEC/Results/BAR302_aftercond/sexc",
    # ]

else:
    print("error in input code for condition. should be HIM-5 or BAR302")


##true other parameters are here

# enter framerate (15fps)
framerate = 15

# max frames missing - max frames that can be missing inside rev and still consider it one reversal instead of 2 (also for forward etc)
# should be 1 sec and is 15fps so 15 frames
max_frames_missing = 15

# min_good_frames_percent - maximum frames a worm can be missing to be considered for analysis - e.g. if is 30 percent worms that have less than 30% good frames excluded
min_good_frames_percent = 30

# min and max length for considering as reversals
minrevlen = 15
maxrevlen = 80

# min and max omega length
minomegalength = 45
maxomegalength = 400

## Actual code to call functions is here 📞

In [20]:
# set some variables
conds = ["mock", "avsv", "sexc"]  # condition codes to append as needed
features_filename = "metadata_featuresN.hdf5"

pathnames_dict = dict()
# loop to get a dictionary containing the  pathnames for each condition
for f in range(0, len(folders)):
    this_path_list = get_file_paths(folders[f], features_filename)
    pathnames_dict[conds[f]] = this_path_list

del f
del this_path_list


# now have pathnames dict, need to reorganise data and keep only the bits we want
data_df, data_dict = organise_data(pathnames_dict)


# create unique ids for each worm and save xls with equivalency to worm video (uniquelongid)
data_df["unique_id_long"] = data_df["filename"] + data_df["worm_id"].astype(str)
newids = data_df["unique_id_long"].unique()
df_newids = pd.Series(newids, name="unique_id_long").reset_index()
df_newids = df_newids.rename(columns={"index": "unique_id"})
data_df = data_df.merge(df_newids, on="unique_id_long")
df_newids.to_excel(outputdir + "worm_unique_id_" + code + ".xlsx")


###################################################

# FOR NOW DONT RUN THIS CAUSE LOOSE LOTS OF WORMS
# # ctrue function to remove worms with few frames
# sheared_data_df = remove_shitty_worms(data_df)
# true_worms_data_df = data_df.copy()
# data_df = sheared_data_df.copy()
# del sheared_data_df

/tmp/ipykernel_11469/2802986360.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  bigdict[pathname_dict[key][i]]["data"]["cond"] = key
/tmp/ipykernel_11469/2802986360.py:23: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  bigdict[pathname_dict[key][i]]["data"]["filename"] = pathname_dict[key][i]
/tmp/ipykernel_11469/2802986360.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.conc

### call  find/clean reversals, forward runs, and omegas functions📞↩️

In [21]:
# to run reversals AND forward run functions
# create list with unique wormlist
wormlist = data_df["unique_id"].unique()


# will contain rev len lists and unique id list matching length to create all_rev_lengths
revlengs_ids = []
revlengs = []

# will contain fwd run len lists and unique id list matching length to create true_fwd_lengths
fwdlengs_ids = []
fwdlengs = []


# add columns to contain fwd and rev start and end indices
data_df["RevStart"] = False
data_df["RevEnd"] = False
data_df["FwdStart"] = False
data_df["FwdEnd"] = False


for worm in wormlist:
    print("this worm: ", worm)
    this_worm_speed = data_df[["frame_number", "speed"]][data_df["unique_id"] == worm]

    # find and clean reversals
    this_worm_RS_dirty, this_worm_RE_dirty = findreversals(this_worm_speed)
    this_worm_RS, this_worm_RE, this_worm_RLg = cleanreversals(
        this_worm_speed, this_worm_RS_dirty, this_worm_RE_dirty
    )

    # find and clean fwd runs
    this_worm_FS_dirty, this_worm_FE_dirty = findfwdruns(this_worm_speed)
    this_worm_FS, this_worm_FE, this_worm_FLg = cleanfwdruns(
        this_worm_speed, this_worm_FS_dirty, this_worm_FE_dirty
    )

    # to store reversal and fwd run lengths
    # cycle thrugh list of reversal/fwdrun lengths and append worm id to worm id list and rev/fwd leng to rev/fwd list
    # then will join both these lists into df
    for k in this_worm_RLg:
        revlengs_ids.append(worm)
        revlengs.append(k)

    for m in this_worm_FLg:
        fwdlengs_ids.append(worm)
        fwdlengs.append(m)

    # now also create columns in data_df that contain true at reversal start/end points (clean)
    data_df.loc[this_worm_RS, "RevStart"] = True
    data_df.loc[this_worm_RE, "RevEnd"] = True
    data_df.loc[this_worm_FS, "FwdStart"] = True
    data_df.loc[this_worm_FE, "FwdEnd"] = True

# thisdf will contain unique worm id (short) as 1st column and the length of each reversal in 2nd column, another df same for fwdruns
all_rev_lengths = pd.DataFrame()
all_rev_lengths = all_rev_lengths.assign(unique_id=revlengs_ids, revlen=revlengs)

all_fwd_lengths = pd.DataFrame()
all_fwd_lengths = all_fwd_lengths.assign(unique_id=fwdlengs_ids, fwdlen=fwdlengs)


# now get rid of ONLY REVERSALS that are too long or too short (dont apply this to forward runs!)
# then rename the the old dataframe so still have access, but use new_data_df as data_df

new_data_df, true_rev_lengths = exclude_short_long_revs(data_df)
data_df_all_revs = data_df.copy()
del data_df
data_df = new_data_df.copy()
del new_data_df

/tmp/ipykernel_11469/2930054385.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data_df["RevStart"] = False
/tmp/ipykernel_11469/2930054385.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data_df["RevEnd"] = False
/tmp/ipykernel_11469/2930054385.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.co

this worm:  0
this worm:  1
this worm:  2
this worm:  3
this worm:  4
this worm:  5
this worm:  6
this worm:  7
this worm:  8
this worm:  9
this worm:  10
this worm:  11
this worm:  12
this worm:  13
this worm:  14
this worm:  15
this worm:  16
this worm:  17
this worm:  18
this worm:  19
this worm:  20
this worm:  21
this worm:  22
this worm:  23
this worm:  24
this worm:  25
this worm:  26
this worm:  27


In [22]:
# add is fwd or is bck column to data_df
data_df["fw"] = False
data_df["bk"] = False

for row in range(len(data_df)):
    if data_df.loc[row, "speed"] > 0:
        data_df.loc[row, "fw"] = True
    elif data_df.loc[row, "speed"] < 0:
        data_df.loc[row, "bk"] = True


# count good frames
wormlist = data_df["unique_id"].unique()
gf_per_worm = count_good_frames(data_df)

gf_per_worm


# cal function to calculate percent time fwd and back
fwdbck = calculate_fbpercent(gf_per_worm, data_df)
fwdbck

fwdbck.to_excel(outputdir + "fwdbck" + code + ".xlsx")

In [41]:
data_df[(data_df["unique_id"] == 19) & (data_df["ogRevStart"] == True)]

,worm_id,has_skeleton,frame_number,x,y,speed,angular_velocity,relative_to_body_speed_midbody,relative_to_body_radial_velocity_head_tip,relative_to_body_angular_velocity_head_tip,...,unique_id_long,unique_id,ogRevStart,ogRevEnd,FwdStart,FwdEnd,RevStart,RevEnd,fw,bk
65534,1.0,1.0,1712.0,2357.853516,1609.601440,-8.048562,0.093520,-10.875153,-72.958847,-0.247894,...,/home/Documents/neuroUCL/phd/current/project/t...,19,True,False,False,False,False,False,False,True
65613,1.0,1.0,1791.0,2353.927734,1602.172363,-22.600433,0.225060,NaN,NaN,NaN,...,/home/Documents/neuroUCL/phd/current/project/t...,19,True,False,False,False,True,False,False,True
65663,1.0,1.0,1841.0,2350.437744,1617.131348,-9.490545,0.196564,NaN,NaN,NaN,...,/home/Documents/neuroUCL/phd/current/project/t...,19,True,True,False,False,False,False,False,True
65680,1.0,1.0,1858.0,2352.784668,1610.534424,-97.581970,0.169504,NaN,NaN,NaN,...,/home/Documents/neuroUCL/phd/current/project/t...,19,True,False,False,False,True,False,False,True
66059,1.0,1.0,2237.0,2354.351318,1552.182007,-22.771564,0.013324,NaN,NaN,NaN,...,/home/Documents/neuroUCL/phd/current/project/t...,19,True,False,False,False,True,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72715,1.0,1.0,8893.0,2744.925293,1493.345947,-15.809999,-0.032908,3.601770,-7.211644,-0.061333,...,/home/Documents/neuroUCL/phd/current/project/t...,19,True,False,False,False,True,False,False,True
72755,1.0,1.0,8933.0,2742.832520,1486.052368,-1.233408,-0.003970,-0.551683,1.916762,0.001126,...,/home/Documents/neuroUCL/phd/current/project/t...,19,True,False,False,False,False,False,False,True
72804,1.0,1.0,8982.0,2745.799072,1491.856445,-8.543286,0.012822,1.328621,-14.043110,0.127997,...,/home/Documents/neuroUCL/phd/current/project/t...,19,True,False,False,False,False,False,False,True
72809,1.0,1.0,8987.0,2745.691162,1491.999268,-1.330577,-0.020607,-1.217804,0.332716,-0.016660,...,/home/Documents/neuroUCL/phd/current/project/t...,19,True,False,False,False,False,False,False,True


In [39]:
all_rev_lengths[all_rev_lengths["unique_id"] == 19]

,unique_id,revlen
221,19,2.0
222,19,21.0
223,19,1.0
224,19,35.0
225,19,17.0
...,...,...
323,19,37.0
324,19,4.0
325,19,2.0
326,19,2.0


In [36]:
sum(all_rev_lengths["revlen"][all_rev_lengths["unique_id"] == 19])

1586.0